<a href="https://colab.research.google.com/github/DanMatsu0/machine-learning-projectsV1/blob/main/mnist_cnn_tensorflow_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [ ]:
import tensorflow as tf

データセットの読み込み

In [ ]:
# データセットのロード
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# データの前処理
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# 出力結果の固定
tf.random.set_seed(0)

# モデルの定義
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# モデルの訓練
model.fit(x_train, y_train, epochs=5)

# モデルの評価
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/5
1875/1875 [==============================] - 29s 15ms/step - loss: 0.1790 - accuracy: 0.9482
Epoch 2/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0612 - accuracy: 0.9813
Epoch 3/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0438 - accuracy: 0.9863
Epoch 4/5
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0356 - accuracy: 0.9885
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.0360 - accuracy: 0.9887
Test accuracy: 0.9886999726295471
